In [ ]:
%run header.ipynb
print(paths)

dryrun = True

from IPython.display import clear_output
sdsobj = SDS.SDSobj(paths['SDS_TOP'], sds_type='D', format='MSEED')

# Parse lookuptable and convert good CSV files to SDS
lookuptableDF = LLE.removed_unnamed_columns(pd.read_csv(paths['lookuptable']))
lookuptableDF = lookuptableDF.sort_values(by=['starttime'])
print("lookuptableDF = \n", lookuptableDF.columns)
print(lookuptableDF)

In [ ]:
transducersDF = LLE.removed_unnamed_columns(pd.read_csv(paths['transducersCSVfile']))
print("transducersDF = \n",transducersDF.columns)

lookuptableDF['SDS'] = False

sds_column_exists = False
if 'SDS' in lookuptableDF.columns:
    sds_column_exists = True

for index, row in lookuptableDF.iterrows():
    print(row)
    row_to_sds_done = False
    if row['passed']: 
        successful = False
        if sds_column_exists:
            row_to_sds_done = row['SDS']
        if not row_to_sds_done:
            df2 = pd.read_csv(os.path.join(paths['CORRECTED'],row['outputfile']))
            print(f"- processing {row['outputfile']} to SDS")
            print("row = \n ",row)
            #print("sdsobj = \n", sdsobj)
            if row['passed']:
                successful = LLE.convert2sds(df2, sdsobj, transducersDF, dryrun=dryrun)
            else:
                try:
                    print('SCAFFOLD: running convert2sds_badfile')
                    successful = LLE.convert2sds_badfile(df2, sdsobj, transducersDF, dryrun=dryrun)
                except:
                    print('Failed to convert bad CSV file')
            lookuptableDF.at[index,'SDS'] = successful
            if successful and not dryrun:
                lookuptableDF.to_csv(lookuptable)     
    clear_output(wait=True)
del sdsobj